In [ ]:
import os
import tensorflow as tf
from keras.utils import to_categorical
import glob
import cv2
import numpy as np
import time
import inspect

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
VGG_MEAN = [103.939, 116.779, 123.68]


def load_data():
    dirs = ['cat', 'lion', 'tiger', 'leopard']
    lab = np.concatenate((np.tile(0, 50), np.tile(1,50), np.tile(2,50), np.tile(3,50)), axis = 0)
    data_train = []
    data_test = []

    for dir_ in dirs:
        files_train = glob.glob(os.path.join('data','train',dir_,'*.jpg'))
        files_test = glob.glob(os.path.join('data','test',dir_,'*.jpg'))
        for train, test in zip(files_train, files_test):
            data_train.append(cv2.imread(train, 1))
            data_test.append(cv2.imread(test, 1))

    labels = to_categorical(lab)
    return np.array(data_train), labels, np.array(data_test), labels

In [ ]:
class Vgg19():
    def __init__(self, vgg19_npy_path=None):
        try:
            self.data_dict = np.load('vgg19.npy', encoding='latin1').item()
        except:
            print("Place vgg19.npy in the same directory as of this notebook.")
            print("You can get it from ")
        self.isTrain = True
        self.epochs = 50
        self.batch_size = 10
        self.data_size = 400
        self.restore = False
        self.decay = 0.99
        self.model_name = 'model'
        self.ckpt_dir = './' + self.model_name + "/checkpoint"
        self.test_fraction = 0.5
        
    def model(self):
        """
        load variable from npy to build the VGG

        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """

        with tf.variable_scope('Input') as scope:   
            self.rgb = tf.placeholder(name = "Input", shape = [None, 224, 224, 3], dtype = tf.float32)
            self.labels = tf.placeholder(name = "Labels", shape = [None, 4], dtype = tf.float32)
            self.isTrain = tf.placeholder(name = "isTrain", shape = None, dtype = tf.bool)
            self.step = tf.train.get_or_create_global_step()
            self.lr = tf.placeholder(name = "LR", shape = None, dtype = tf.float32)

        # Convert RGB to BGR
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=self.rgb)
        bgr = tf.concat(axis=3, values=[
            blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
            red - VGG_MEAN[2],
        ])
        
        self.conv1_1 = self.conv_layer(bgr, "conv1_1")
        self.conv1_2 = self.conv_layer(self.conv1_1, "conv1_2")
        self.pool1 = self.max_pool(self.conv1_2, 'pool1')

        self.conv2_1 = self.conv_layer(self.pool1, "conv2_1")
        self.conv2_2 = self.conv_layer(self.conv2_1, "conv2_2")
        self.pool2 = self.max_pool(self.conv2_2, 'pool2')

        self.conv3_1 = self.conv_layer(self.pool2, "conv3_1")
        self.conv3_2 = self.conv_layer(self.conv3_1, "conv3_2")
        self.conv3_3 = self.conv_layer(self.conv3_2, "conv3_3")
        self.conv3_4 = self.conv_layer(self.conv3_3, "conv3_4")
        self.pool3 = self.max_pool(self.conv3_4, 'pool3')

        self.conv4_1 = self.conv_layer(self.pool3, "conv4_1")
        self.conv4_2 = self.conv_layer(self.conv4_1, "conv4_2")
        self.conv4_3 = self.conv_layer(self.conv4_2, "conv4_3")
        self.conv4_4 = self.conv_layer(self.conv4_3, "conv4_4")
        self.pool4 = self.max_pool(self.conv4_4, 'pool4')

        output = self.pool4

        self.conv5_1 = self.conv_layer(self.pool4, "conv5_1")
        self.conv5_2 = self.conv_layer(self.conv5_1, "conv5_2")
        self.conv5_3 = self.conv_layer(self.conv5_2, "conv5_3")
        self.conv5_4 = self.conv_layer(self.conv5_3, "conv5_4")
        self.pool5 = self.max_pool(self.conv5_4, 'pool5')

        self.fc6 = self.fc_layer(self.pool5, "fc6")
        assert self.fc6.get_shape().as_list()[1:] == [4096]
        self.relu6 = tf.nn.relu(self.fc6)

        self.fc7 = self.fc_layer(self.relu6, "fc7")
        self.relu7 = tf.nn.relu(self.fc7)

        self.logits = self.fc_layer_output(self.relu7, "fc8")
        #self.logits = self.fc_layer_output(self.relu6, "fc7")
    
        with tf.variable_scope("Loss") as scope:
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.labels))

        with tf.variable_scope("Accuracy") as scope:

            self.logits_max_args = tf.argmax(self.logits, axis = 1)
            self.labels_max_args = tf.argmax(self.labels, axis = 1)
            self.equal = tf.reduce_sum(tf.cast(tf.equal(self.logits_max_args, self.labels_max_args), tf.float32))
            self.batch_acc = tf.divide(self.equal, tf.cast(tf.shape(self.logits)[0], tf.float32))

        with tf.variable_scope("Optimizer") as scope:

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.loss, self.step)
            self.saver = tf.train.Saver(max_to_keep = 3)
    
    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):
            filt = self.get_conv_filter(name)

            conv = tf.nn.conv2d(bottom, filt, [1, 1, 1, 1], padding='SAME')

            conv_biases = self.get_bias(name)
            bias = tf.nn.bias_add(conv, conv_biases)

            relu = tf.nn.relu(bias)
            return relu

    def fc_layer(self, bottom, name):
        with tf.variable_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])

            weights = self.get_fc_weight(name)
            biases = self.get_bias(name)

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

            return fc

    def fc_layer_output(self, x, name):
        with tf.variable_scope(name):
            shape = x.get_shape().as_list()
            xavier = tf.contrib.layers.xavier_initializer()
            weights = tf.get_variable("Weight", shape=[shape[1], 4], initializer = xavier, dtype = tf.float32)
            biases = tf.get_variable("Bias", shape=[4], initializer = xavier, dtype = tf.float32)

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

            return fc
        
    def get_conv_filter(self, name):
        initializer = self.data_dict[name][0]
        return tf.get_variable("Filter", initializer = initializer, dtype = tf.float32)

    def get_bias(self, name):
        initializer = self.data_dict[name][1]
        return tf.get_variable("Bias", initializer = initializer, dtype = tf.float32)

    def get_fc_weight(self, name):
        initializer = self.data_dict[name][0]
        return tf.get_variable("Weight", initializer = initializer, dtype = tf.float32)
    
    def train(self):
        tf.reset_default_graph()
        print('Loading Data')
        train_data, train_labels, test_data, test_labels = load_data()
        print('Data Loaded')
        
        print('Loading Model')
        self.model()
        print('Model Loaded')
        
        if not os.path.exists(self.model_name):
            os.mkdir(self.model_name)
        
        test_data_size = int(self.data_size * self.test_fraction)
        train_data_size = self.data_size - test_data_size

        train_batches = train_data_size // self.batch_size
        test_batches = test_data_size // self.batch_size

        #data = ((data/255) * 2) - 1 # [0,255] --> [-1,1]
        
        with tf.Session() as self.sess:

           
            init_op = tf.global_variables_initializer()
            self.sess.run(init_op)
            parameter_count = tf.reduce_sum([tf.reduce_prod(tf.shape(v)) for v in tf.trainable_variables()])
            
            print('Parameters:', parameter_count.eval()) 
            start = 0
            acc_best = -1
            lr = 0.0001

            data_perm = np.random.permutation(train_data_size)
            train_perm = data_perm
            test_perm = np.copy(data_perm)

            print('Training commences from epoch ', start)

            for i in range(start, self.epochs):
                
                count = 0
                count_test = 0
                avg_loss = 0

                shuffle = np.random.permutation(train_data_size)
                train_perm = train_perm[shuffle]

                if i+1 == 50:
                    lr = lr/10
                
                for j in range(train_batches):
                    
                    begin = time.time()
                    
                    if j != train_batches-1 :
                        current_batch = train_perm[j*self.batch_size : (j+1)*self.batch_size]
                        x = train_data[current_batch]
                        y = train_labels[current_batch]
                    else:
                        current_batch = train_perm[j*self.batch_size: ]
                        x = train_data[current_batch]
                        y = train_labels[current_batch]

                    feed_train = {self.rgb: x, self.labels: y, self.lr: lr}
                    
                    _, eq, loss = self.sess.run([self.optimizer, self.equal, self.loss], feed_dict = feed_train)
                    count += eq
                    avg_loss += loss
                    
                    line = 'Batch: %d Batch Accuracy: %.4f Loss: %.4f Time/Batch: %.4f' %(j, eq/len(current_batch), float(loss), time.time() - begin)
                    print(line, end ='\r')

                for j in range(test_batches):
                    
                    print('======================Testing====================', end = '\r')
                    begin = time.time()
                    
                    if j != test_batches-1 : 
                        current_batch = test_perm[j*self.batch_size: (j+1)*self.batch_size]
                        x = test_data[current_batch]
                        y = test_labels[current_batch]
                    else:
                        current_batch = test_perm[j*self.batch_size: ]
                        x = test_data[current_batch]
                        y = test_labels[current_batch]

                    feed_test = {self.rgb: x, self.labels: y}
                    eq = self.sess.run(self.equal, feed_dict = feed_test)
                    #print(eq)
                    count_test += eq
                
                accuracy = count / train_data_size
                accuracy_test = count_test / test_data_size
                avg_loss = avg_loss / train_batches

                line = "Epoch: %d Train Acc: %.6f Test Acc: %.6f Average Loss/Batch: %.6f" %(i,accuracy,accuracy_test,avg_loss)
                print(line)

                with open(os.path.join(self.model_name,'logs.txt'), 'a') as f :
                    line += '\n'
                    f.write(line)

In [ ]:
net = Vgg19()
net.train()